# Problem Statement Given

The candidate needs to make an algorithm for auto indicator turning off making any assumptions on the ride on the scooter. Please ensure the candidates considers all situations when the indicator is used (needs to come up with all assumptions)

## Evaluation Criteria

The results will be evaluated on the basis of -

1. Clearly stating the problem he is trying to solve.
2. Understanding the problem.
3. Creativity.
4. The difficulty of the Algorithm.
5. Coding standard.
6. Readable Code.

## Expected output
A practical Algorithm that can be used in a real-world scenario

# My Solution to the Problem Statement

## 1. Importing necessary modules and libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime

## 2. Load Dataset

In [ ]:
# Loading the CSV file
data = pd.read_csv("ScooterIMUData.csv")

print("Shape of Data : ",data.shape)
data.head()

Shape of Data :  (334467, 5)


,ts,received_ts,device_uuid,data_item_name,value
0,2021-11-10 12:18:47.150,2021-11-10 12:18:56.856,s_5777,GYR_X_DEG,-0.010051
1,2021-11-10 12:18:47.150,2021-11-10 12:18:56.860,s_5777,GYR_Y_DEG,-0.076319
2,2021-11-10 12:18:47.150,2021-11-10 12:18:56.865,s_5777,GYR_Z_DEG,-0.044205
3,2021-11-10 12:18:47.170,2021-11-10 12:18:56.856,s_5777,GYR_X_DEG,-0.029533
4,2021-11-10 12:18:47.170,2021-11-10 12:18:56.860,s_5777,GYR_Y_DEG,-0.014959


## 3. Cleaning and Pre-processing of Data

In [ ]:
# Dropping any NaN values ( if any )
print("Before dropping NaN rows : ",data.shape[0])
data = data.dropna()
print("After dropping NaN rows : ",data.shape[0])

# Dropping the device_uuid column ( OPTIONAL )
data = data.drop('device_uuid',axis=1)
print("After dropping device_uuid column : ",data.shape)

Before dropping NaN rows :  334467
After dropping NaN rows :  334467
After dropping device_uuid column :  (334467, 4)


In [ ]:
# Getting time from date - time format
def get_time(x):
  return x.split()[1]

# Cleaning the Gyro Axis Label to X,Y and Z
def get_gyro_axis(x):
  return x.split("_")[1]

# Apply the get_time function to both time related columns
data['ts'] = data['ts'].apply(get_time)
data['received_ts'] = data['received_ts'].apply(get_time)

# Clean labels for the data_item_name column
data['data_item_name'] = data['data_item_name'].apply(get_gyro_axis)

data.head()

,ts,received_ts,data_item_name,value
0,12:18:47.150,12:18:56.856,X,-0.010051
1,12:18:47.150,12:18:56.860,Y,-0.076319
2,12:18:47.150,12:18:56.865,Z,-0.044205
3,12:18:47.170,12:18:56.856,X,-0.029533
4,12:18:47.170,12:18:56.860,Y,-0.014959


In [ ]:
# Removing the Z axis reading - Assuming that the scooter will not move in that direction ( upwards direction )
data = data[data.data_item_name != "Z"]

# Resetting indexing and dropping the extra column that comes due to reset_index
data.reset_index(inplace = True)
data = data.drop('index',axis=1)

data.head()

,ts,received_ts,data_item_name,value
0,12:18:47.150,12:18:56.856,X,-0.010051
1,12:18:47.150,12:18:56.860,Y,-0.076319
2,12:18:47.170,12:18:56.856,X,-0.029533
3,12:18:47.170,12:18:56.860,Y,-0.014959
4,12:18:47.190,12:18:56.856,X,0.016557


In [ ]:
# Converting string to date time object - to compute time difference easily

def convert_2_time(x):
  return datetime.datetime.strptime(x, "%H:%M:%S.%f")

# Apply the function to both the time related columns
data['ts'] = data['ts'].apply(convert_2_time)
data['received_ts'] = data['received_ts'].apply(convert_2_time)

data.head()

,ts,received_ts,data_item_name,value
0,1900-01-01 12:18:47.150,1900-01-01 12:18:56.856,X,-0.010051
1,1900-01-01 12:18:47.150,1900-01-01 12:18:56.860,Y,-0.076319
2,1900-01-01 12:18:47.170,1900-01-01 12:18:56.856,X,-0.029533
3,1900-01-01 12:18:47.170,1900-01-01 12:18:56.860,Y,-0.014959
4,1900-01-01 12:18:47.190,1900-01-01 12:18:56.856,X,0.016557


## 4. Computing Delta Time

In [ ]:
# add a delta time column to the data frame
data['delta_time'] = [
                      (data['received_ts'][i] - data['ts'][i]).total_seconds()
                      for i in range(data.shape[0])
                      ]

data.head()

,ts,received_ts,data_item_name,value,delta_time
0,1900-01-01 12:18:47.150,1900-01-01 12:18:56.856,X,-0.010051,9.706
1,1900-01-01 12:18:47.150,1900-01-01 12:18:56.860,Y,-0.076319,9.710
2,1900-01-01 12:18:47.170,1900-01-01 12:18:56.856,X,-0.029533,9.686
3,1900-01-01 12:18:47.170,1900-01-01 12:18:56.860,Y,-0.014959,9.690
4,1900-01-01 12:18:47.190,1900-01-01 12:18:56.856,X,0.016557,9.666


## 5. Compute Roll and Pitch Angle 

In [ ]:
# Initally both roll and pitch angles are taken as 0
Gyro_Roll = 0
Gyro_Pitch = 0

# Computed roll for X axis movements and pitch angle for Y axis movements for all the data values
Gyro_Roll_list = []
Gyro_Pitch_list = []
for i in range(data.shape[0]):
  if(data['data_item_name'][i]=="X"):
    val = Gyro_Roll - data['value'][i]*data['delta_time'][i]
    Gyro_Roll_list.append(val)
    Gyro_Roll = val
    Gyro_Pitch_list.append(0)
  
  elif(data['data_item_name'][i]=="Y"):
    val = Gyro_Pitch + data['value'][i]*data['delta_time'][i]
    Gyro_Pitch_list.append(val)
    Gyro_Pitch = val
    Gyro_Roll_list.append(0)
  
  else:
    Gyro_Pitch_list.append(0)
    Gyro_Roll_list.append(0)

# Added the angle values to the dataframe
data["Gyro_Pitch"] = Gyro_Pitch_list
data["Gyro_Roll"] = Gyro_Roll_list

data.head()

,ts,received_ts,data_item_name,value,delta_time,Gyro_Pitch,Gyro_Roll
0,1900-01-01 12:18:47.150,1900-01-01 12:18:56.856,X,-0.010051,9.706,0.000000,0.097556
1,1900-01-01 12:18:47.150,1900-01-01 12:18:56.860,Y,-0.076319,9.710,-0.741062,0.000000
2,1900-01-01 12:18:47.170,1900-01-01 12:18:56.856,X,-0.029533,9.686,0.000000,0.383616
3,1900-01-01 12:18:47.170,1900-01-01 12:18:56.860,Y,-0.014959,9.690,-0.886011,0.000000
4,1900-01-01 12:18:47.190,1900-01-01 12:18:56.856,X,0.016557,9.666,0.000000,0.223573


## 6. Compute Angle of Inclination

In [ ]:
all_inclination = []
for i in range(data.shape[0]):
  val1 = np.sqrt(np.power(np.tan(data['Gyro_Roll'][i]),2) + np.power(np.tan(data['Gyro_Pitch'][i]),2))
  val2 = np.degrees(np.arctan(val1))
  all_inclination.append(val2)

data['Inclination Angle'] = all_inclination

data.head()

,ts,received_ts,data_item_name,value,delta_time,Gyro_Pitch,Gyro_Roll,Inclination Angle
0,1900-01-01 12:18:47.150,1900-01-01 12:18:56.856,X,-0.010051,9.706,0.000000,0.097556,5.589519
1,1900-01-01 12:18:47.150,1900-01-01 12:18:56.860,Y,-0.076319,9.710,-0.741062,0.000000,42.459716
2,1900-01-01 12:18:47.170,1900-01-01 12:18:56.856,X,-0.029533,9.686,0.000000,0.383616,21.979599
3,1900-01-01 12:18:47.170,1900-01-01 12:18:56.860,Y,-0.014959,9.690,-0.886011,0.000000,50.764679
4,1900-01-01 12:18:47.190,1900-01-01 12:18:56.856,X,0.016557,9.666,0.000000,0.223573,12.809770


## 7. Predict Indicator Signal

***Assuming that our gyrometer is on the right hand side.***

1. Roll Angle

If negative - Left - Roll towards left

If positive - Right - Roll towards right

2. Pitch Angle

If negative - Right

If positive - Left

In [ ]:
indicator = []
for i in range(data.shape[0]):
  gp = data['Gyro_Pitch'][i]
  gr = data['Gyro_Roll'][i]

  if(gr == 0 and gp ==0):
    indicator.append("OFF")
  elif( gp ==0):
    if(gr<0):
      indicator.append("LEFT")
    else:
      indicator.append("RIGHT")
  else:
    if(gp>0):
      indicator.append("LEFT")
    else:
      indicator.append("RIGHT")

data["Indicator"] = indicator
data.head()

,ts,received_ts,data_item_name,value,delta_time,Gyro_Pitch,Gyro_Roll,Inclination Angle,Indicator
0,1900-01-01 12:18:47.150,1900-01-01 12:18:56.856,X,-0.010051,9.706,0.000000,0.097556,5.589519,RIGHT
1,1900-01-01 12:18:47.150,1900-01-01 12:18:56.860,Y,-0.076319,9.710,-0.741062,0.000000,42.459716,RIGHT
2,1900-01-01 12:18:47.170,1900-01-01 12:18:56.856,X,-0.029533,9.686,0.000000,0.383616,21.979599,RIGHT
3,1900-01-01 12:18:47.170,1900-01-01 12:18:56.860,Y,-0.014959,9.690,-0.886011,0.000000,50.764679,RIGHT
4,1900-01-01 12:18:47.190,1900-01-01 12:18:56.856,X,0.016557,9.666,0.000000,0.223573,12.809770,RIGHT


The Final column indicates the situation of Indicator at a particular moment.